In [1]:
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

options = ChromeOptions()
options.add_argument('--headless')
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox") 
options.binary_location = '/usr/bin/chromium-browser'
driver = webdriver.Chrome(options=options)

In [2]:
from src import const,utils,sqlite
hashtag = '#絵かゆ'
date = 14

driver = utils.get_driver(False)
url = utils.get_url(hashtag)
driver.get(url)


CHROME >= 115.0.5763.0, using mac-arm64 as architecture identifier


In [3]:
import datetime
import re

# 入力文字列
#input_string = "11分前"  # または "9月9日(土) 22:28"
def date_parse(input_string:str):
    # 現在の日時を取得
    current_time = datetime.datetime.now()

    if ("秒前" in input_string) and ("昨日" not in input_string):
        # "分前" を抽出して分数に変換
        second_ago = int(input_string.split("秒前")[0])
        
        # 指定された分数だけ過去の日時を計算
        new_time = current_time - datetime.timedelta(seconds=second_ago)

    elif ("分前" in input_string) and ("昨日" not in input_string):
        # "分前" を抽出して分数に変換
        minutes_ago = int(input_string.split("分前")[0])
        
        # 指定された分数だけ過去の日時を計算
        new_time = current_time - datetime.timedelta(minutes=minutes_ago)
    elif "昨日" in input_string:
        output_string = input_string.replace('昨日 ','')
        #　昨日の日時を設定
        time_parts = output_string.split(":")
        new_time = current_time.replace(hour=int(time_parts[0]), minute=int(time_parts[1]), second=0)
        #new_time = new_time + datetime.timedelta(days=-1)

    elif (":" in input_string) and ('月' not in input_string):
        # "16:15" のような場合、今日の日付に指定時刻を設定
        time_parts = input_string.split(":")
        new_time = current_time.replace(hour=int(time_parts[0]), minute=int(time_parts[1]), second=0)
    else:
        # "9月9日(土) 22:28" のような場合、指定されたフォーマットで日時を解析 9月9日(土) 22:28
        output_string = re.sub(r'\([^)]*\)', '', input_string)
        new_time = datetime.datetime.strptime(output_string, "%m月%d日 %H:%M")
        new_time = new_time.replace(year=current_time.year)
    return new_time

In [5]:
from bs4 import BeautifulSoup
from tqdm import tqdm
#Import-------------------------------------------------------------------------------
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
from urllib import request
import requests
import chromedriver_autoinstaller
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.service import Service
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from urllib.parse import quote

#-------------------------------------------------------------------------------------
# 現在の日時を取得
now = datetime.datetime.now()
date_time = datetime.datetime.now()
tweets = []
load_flag = 3
load_now = 0
while(True):

    #ツイートのリストを取得
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tweet_links = soup.find_all('div', class_='Tweet_bodyContainer__n_Cs6')#Tweet_bodyContainer__n_Cs6

    #ツイートから各要素を取得

    for tweet in tqdm(tweet_links):
        url = ''
        try:
            url = [link for link in tweet.find_all('a') if 'pic.twitter.com' in link.text][0].text#ツイートリンク
        except:
            url = ''
        try:
            images = [i['src'] for i in [link for link in tweet.find('div','Tweet_imageContainerWrapper__wPE0R').find_all('img') if 'https://rts-pctr.c.yimg.jp' in link['src']]]#画像のリスト
        except:
            images = []
        like_count = 0#いいねっ
        try:
            like_count = int([a for a in tweet.find_all('a') if 'like' in a.get('href','')][0].find('span').text.replace(',',''))
        except:
            like_count = 0
        user_name = tweet.find('a','Tweet_authorID__B1U8c').text#ユーザーID
        display_name = tweet.find('span','Tweet_authorName__V3waK').text#ユーザー名
        
        date_text = tweet.find('time').text
        date_time = date_parse(date_text)

        if like_count >= 10 and len(images) > 0 and len([t for t in tweets if url == t[0]]) == 0 and url != '':
            tweets.append([
                url,
                images,
                date_time,
                user_name,
                display_name,
                like_count
            ])
    print(f'tweet count -> {len(tweets)}')
    if (now - date_time).days > date:
        print('datetime',date_time)
        print(f'date over => {(now - date_time).days}/{date}')
        break

    #一番下までスクロール
    for ind in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.25)
    sleep(3)
    try:
        load_btn = driver.find_element(By.LINK_TEXT,"もっと見る")
        load_btn.click()
    except:
        load_now = load_now + 1
        
    if load_now >= load_flag:
        print(f'button over => {load_now} / {load_flag}')
        break
    

for t in tweets:
    print(t)


100%|██████████| 40/40 [00:00<00:00, 8108.46it/s]


tweet count -> 25


100%|██████████| 40/40 [00:00<00:00, 9762.71it/s]


tweet count -> 25


100%|██████████| 50/50 [00:00<00:00, 9626.59it/s]


...

tweet count -> 239


100%|██████████| 340/340 [00:00<00:00, 9589.62it/s]


tweet count -> 248


100%|██████████| 350/350 [00:00<00:00, 8954.59it/s]

tweet count -> 252
datetime 2023-10-05 22:21:00
date over => 15/14
['pic.twitter.com/qm2Ulta5UG', ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-1MeLS8o2Cw3sK-96faaLlVxFQm3DqfkzDdw2BZcHAwubl67WAYV8bt-L87FVGyqTntdsd2pNuT9I3lwGYVjLDgHxAe-aE8edVKVCbJQ7uCvjzKmjgrs26_DtMjErSZdP'], datetime.datetime(2023, 10, 21, 21, 15, 0, 580649), '@I_love_micochi', 'またたび@手描き切り抜き', 13]
['pic.twitter.com/JgM1dZh5FX', ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-Y8Z3LVDcAYN0jZz-uJLAzEHdxIyG2u6FQks4DMDBd_-4j2SVDir7UZnF1hFfEdYP5mf2rH634LGbE03lbXAj0jfbSj6sYmDgeJXv8zTgy2smxHogs0iwrhG72AM77_5U'], datetime.datetime(2023, 10, 21, 20, 35, 0, 580760), '@Cabeico', 'ガベ子 🖍 嘎唄.Art', 88]
['pic.twitter.com/wc52o5BlBK', ['https://rts-pctr.c.yimg.jp/BgIFgYJGVIVv8u0nVJvw8B2FauAYMF4jOP5zFJAV4i6hjVpND1OjWZ7RVcmujne-YSJ0f9PtnGAUaWDDDbZhiXqq8XDLpH466W4hcj4CuAcYW-sv2rt6iG84FJBVelX1R_A0K20tSMBmvBVI87v7LN7BR8Rj0bGlqgzEyiJREIv5XJAheIiO5qlf2Y

In [4]:
input_string = '9月10日(日) 17:01'
output_string = re.sub(r'\([^)]*\)', '', input_string)
print(output_string)

9月10日 17:01


In [20]:
import os
import pandas as pd


base_path = './'
holo_path = os.path.join(base_path, "options", "HoloFanArt.csv")
df = pd.read_csv(holo_path)
records = []
for index, row in df.iterrows():
    r = row
    hashtag = row["hashtag"]
    url = row["url"]
    rec = HoloName(hashtag,url)
    records.append(rec)
print(row)

NameError: name 'HoloName' is not defined

In [16]:
print(row)

url    https://hololive.hololivepro.com/wp-content/up...
Name: #soraArt, dtype: object


In [13]:
print(row['url'])

https://hololive.hololivepro.com/wp-content/uploads/2021/05/tokino_sora_thumb.png


In [19]:
df

,url
hashtag,
#soraArt,https://hololive.hololivepro.com/wp-content/up...
#ロボ子Art,https://hololive.hololivepro.com/wp-content/up...
#miko_Art,https://hololive.hololivepro.com/wp-content/up...
#ほしまちぎゃらりー,https://hololive.hololivepro.com/wp-content/up...
#AZKiART,https://hololive.hololivepro.com/wp-content/up...
...,...
#Artnose,https://hololive.hololivepro.com/wp-content/up...
#ririkart,https://hololive.hololivepro.com/wp-content/up...
#らでん似絵,https://hololive.hololivepro.com/wp-content/up...
